In [1]:
import pandas as pd
from sklearn.metrics import classification_report


df1 = pd.read_csv('results_mobilnet.csv')  
df2 = pd.read_csv('results_my_model.csv')  

report1 = classification_report(df1['true_labels'], df1['predicted_labels'], output_dict=True)
report2 = classification_report(df2['true_labels'], df2['predicted_labels'], output_dict=True)

report_df1 = pd.DataFrame(report1).transpose()
report_df2 = pd.DataFrame(report2).transpose()

comparison = pd.concat([report_df1, report_df2], axis=1, keys=['Model1', 'Model2'])
print(comparison)


                Model1                                   Model2            \
             precision    recall  f1-score    support precision    recall   
0             0.962963  1.000000  0.981132  26.000000  0.925926  0.961538   
1             0.928571  0.650000  0.764706  20.000000  0.923077  0.600000   
2             0.720000  0.900000  0.800000  20.000000  0.692308  0.900000   
accuracy      0.863636  0.863636  0.863636   0.863636  0.833333  0.833333   
macro avg     0.870511  0.850000  0.848613  66.000000  0.847104  0.820513   
weighted avg  0.878916  0.863636  0.860660  66.000000  0.854269  0.833333   

                                   
              f1-score    support  
0             0.943396  26.000000  
1             0.727273  20.000000  
2             0.782609  20.000000  
accuracy      0.833333   0.833333  
macro avg     0.817759  66.000000  
weighted avg  0.829181  66.000000  
